# Cleaning string data
Data cleaning is the process of fixing or removing incorrect, corrupted, incorrectly formatted, duplicate, or incomplete data within a dataset.
For strings this process starts by applying a list of common functions (lower, strip).  Then handling missing and duplicate entries. And finally applying custom functions as needed.

In [43]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import pandas as pd
import numpy as np

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Set max rows and columns displayed in jupyter
# pd.set_option("display.max_rows", 100)
# pd.set_option("display.max_columns", 20)

## Load a dataset

In [44]:
FILE_RAW = '../projects/proj1/data/Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.csv'
FILE_FEATHER = '../projects/proj1/data/AskAManagerSalarySurvey2021-Form Responses1.feather'

In [45]:
df=pd.read_csv(FILE_RAW)
df.head()
print(df.shape)

,Timestamp,How old are you?,Industry,Job title,Additional context on job title,Annual salary,Other monetary comp,Currency,Currency - other,Additional context on income,Country,State,City,Overall years of professional experience,Years of experience in field,Highest level of education completed,Gender,Race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


(27609, 18)


In [46]:
# what type is country?  if object then it's a string
df.dtypes

Timestamp                                    object
How old are you?                             object
Industry                                     object
Job title                                    object
Additional context on job title              object
Annual salary                                object
Other monetary comp                         float64
Currency                                     object
Currency - other                             object
Additional context on income                 object
Country                                      object
State                                        object
City                                         object
Overall years of professional experience     object
Years of experience in field                 object
Highest level of education completed         object
Gender                                       object
Race                                         object
dtype: object

In [47]:
x=df['Country'].unique()
x.sort()
x[:50]

array([' New Zealand', ' U.S.', ' US', ' United States',
       '$2,175.84/year is deducted for benefits',
       'ARGENTINA BUT MY ORG IS IN THAILAND', 'Afghanistan', 'Africa',
       'America', 'Aotearoa New Zealand', 'Argentina', 'Australi',
       'Australia', 'Australia ', 'Australian ', 'Austria',
       'Austria, but I work remotely for a Dutch/British company',
       'Bangladesh', 'Bangladesh ', 'Belgium', 'Belgium ', 'Bermuda',
       'Brasil', 'Brazil', 'Brazil ', 'Britain ', 'Bulgaria', 'CANADA',
       'CANADA ', 'California ', 'Cambodia', 'Can', 'Canad', 'Canada',
       'Canada ', 'Canada and USA', 'Canada, Ottawa, ontario', 'Canadw',
       'Canadá', 'Canda', 'Catalonia', 'Cayman Islands', 'Chile', 'China',
       'Colombia', 'Company in Germany. I work from Pakistan.', 'Congo',
       'Contracts', 'Costa Rica', "Cote d'Ivoire"], dtype=object)

## Lets handle missing Country data first to ensure below algorithms run correctly.  
Lots of ways to do this, I'm going to do it the simple way, replace NaN with UNKNOWN

This is not a good idea in general, especially if you have a way to figure out the country from the other data present

In [48]:
#how many missing Countries
df.Country.isnull().sum()

0

In [49]:
# None missing, if there were you could fill with a Sentinal value for now, 
# like UNKNOWN, makes it easy to find and deal with later, also ensures that
# calculations do not fail because of NaNs
df.Country = df.Country.fillna('UNKNOWN')

In [50]:
(df.Country=='UNKNOWN').sum()

0

## There is a country column, lets use it to get all the USA entries.  Take a look at the number of unique entries

In [51]:
#a small functions to see the number of unique country names
def p_unique(df=df,col='Country'):print(f'There are now {df[col].nunique()} unique {col} entries')
p_unique()

There are now 364 unique Country entries


In [52]:
#how many different countries are there
start_with_this_many__unique_countries = df.Country.nunique()
p_unique()

There are now 364 unique Country entries


## How many occurrences for each unique entry?

In [53]:
#lets see what we have
vc=df.Country.value_counts()
print(f'There are {len(vc)} unique entries')
vc[:50]
# vc[50:100]

There are 364 unique entries


United States                8844
USA                          7847
US                           2572
Canada                       1549
United States                 652
U.S.                          571
UK                            566
United Kingdom                540
USA                           468
Usa                           441
United States of America      421
Australia                     312
United states                 203
usa                           180
Germany                       168
England                       134
united states                 113
Us                            103
Ireland                       102
New Zealand                    94
Uk                             84
Canada                         75
Australia                      67
United Kingdom                 65
France                         63
U.S.A.                         46
United States of America       43
Spain                          40
Netherlands                    40
Scotland      

## It looks like there was no filtering on what a user could enter in the Country field, ANyway lets get to it

## Apply lower and strip to get the easy gains

In [54]:
df.Country = df.Country.map(str.lower).map(str.strip)

p_unique()

There are now 250 unique Country entries


## Looks like a lot of punctuation, get rid of it all
Use regular expressions

In [55]:
import re

In [56]:
#the regular expressions package
import re
punc = "[!\"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~\`]"  #this is the punctuation to get rid of

#or a function
def fun1(x):
    #re.sub will remove any punction char found in punc 
    return re.sub(punc, '', x)

df.Country = df.Country.map(fun1)

#can do the same thing with a lambda
# df.Country = df.Country.map(lambda x: re.sub(punc, '', x))

p_unique()

There are now 236 unique Country entries


In [57]:
#start here Friday 1/20/22
# type(df.Country.unique())  #what is it?
p_unique()
# np.sort(df.Country.unique())

There are now 236 unique Country entries


In [58]:
#lets see all unique country names that occur at leat twice
ds=df.Country.value_counts()

#filter 1 offs
ds[ds>2]

united states             9842
usa                       9014
us                        3349
canada                    1656
uk                         695
                          ... 
is                           3
taiwan                       3
romania                      3
england united kingdom       3
isa                          3
Name: Country, Length: 68, dtype: int64

In [59]:
#sort by index if you want 
ds1=ds.sort_index()
ds1[:10]  #if you do them all then the really long index entry will knacker the output

#turn index into column, turns series into dataframe
# can access Country now like any other column
# df1=ds.reset_index()
# df1.sort_values(by=['index'])

217584year is deducted for benefits      1
afghanistan                              1
africa                                   1
america                                 21
aotearoa new zealand                     1
argentina                                5
argentina but my org is in thailand      1
australi                                 1
australia                              382
australian                               1
Name: Country, dtype: int64

## Looks like a lot of variations of 'united state'

Another easy gain, lets replace all strings with 'united state' in them with 'usa' 

In [60]:
def fun(x):
    """
    replaces any string that contains 'united state' with 'usa'
    BUT 'united state' and 'usa' are hardcoded, what if we wanted to use other values?
    """
    if 'united state' in x:
        return 'usa'
    return x

#dont do this yet there is a better more general way below
# df.Country = df.Country.map(fun);

## But you might want to do something similar with other strings, do you write another function?  Or do something a little more general?
Be general, always, use a python closure.

## Closures
The problem we face is that map takes a function that takes 1 parameter, and we want it to take 3; The string value passed by map (call it x), the string to search for in x (call it str_to_find), and the string to replace x with if we find str_to_find in x (call it str_replacement).

We can't get around the fact that map only passes 1 parameter to the function.  But we can create a function that  already knows what str_to_find and str_replacement are.  Its called a closure

In [61]:
def fun1(str_to_find, str_replacement):
    """
    creates findandreplace Closure, which is a stateful function
    that remembers str_to_find and str_replacement values
    returns: findandreplace
    """
    def findandreplace(x):
        if str_to_find in x:
            return str_replacement
        return x
    # in python functions are first class objects
    # we are returning findandreplace, it in turn knows the value of 
    # str_to_find and str_replacement and takes an argument to search thru,x
    return findandreplace

#using the closure
fn= fun1('us', 'usa')   #Uh Ohh, what about Australia? Or Austria or Belarus or Cyprus or Russia
                        #careful with this sort of thing!
# df.Country = df.Country.map(fn)

In [62]:
#using the closure one at a time

# fn= fun1('usa', 'usa')
# df.Country = df.Country.map(fn)

# fn= fun1('us', 'usa')
# df.Country = df.Country.map(fn)

# fn= fun1('u s', 'usa')
# df.Country = df.Country.map(fn)

# fn= fun1('unites states', 'usa')  #its a bit suspicious that 17 people made this mistake
# df.Country = df.Country.map(fn)

# fn= fun1('united sates', 'usa')
# df.Country = df.Country.map(fn)

# fn= fun1('unitedstates', 'usa')
# df.Country = df.Country.map(fn)

# fn= fun1('united stares', 'usa')
# df.Country = df.Country.map(fn)
#and so on

In [63]:
#OR you can simplify the above with a list of str_to_find
#and just iterate over it
vals=['usa',  'u s', 'unites states', 'united sates', 'unitedstates', 'united stares', 'united stat', 'america']
for val in vals:
    fn= fun1(val, 'usa')
    df.Country = df.Country.map(fn)
p_unique()

There are now 199 unique Country entries


## Once you get down to the bottom of the unique values you will probably get a lot of one offs
For instance lets see what the the values are

In [66]:
# vals=df.Country.unique()
# vals.sort()
# vals
df.Country.value_counts()[-50:]

mainland china                                                                                                                                                                         1
csnada                                                                                                                                                                                 1
japan us gov position                                                                                                                                                                  1
hartford                                                                                                                                                                               1
new zealand aotearoa                                                                                                                                                                   1
uniteed states                                                             

## Notice there are a lot of united states ish entries at the bottom, lets see if fuzzy wuzzy helps

In [ ]:
# this package lives in the conda forge
# !conda install -c conda-forge fuzzywuzzy -y

In [67]:
# helpful modules
import fuzzywuzzy
from fuzzywuzzy import process

In [68]:
#looking for strings similar to 'unit', for return list
# first is the match, second is the score, third is the index in the dataframe
matches = fuzzywuzzy.process.extract("unit", df.Country, limit=50, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('united y', 67, 22822),
 ('kuwait', 60, 1915),
 ('unite states', 50, 10447),
 ('unite states', 50, 13362),
 ('united sttes', 50, 14113),
 ('united kindom', 47, 7156),
 ('uniter statez', 47, 16890),
 ('unitef stated', 47, 20206),
 ('united  states', 47, 26059),
 ('lithuania', 46, 11426),
 ('lithuania', 46, 20695),
 ('united kingdom', 44, 1),
 ('united kingdom', 44, 15),
 ('united kingdom', 44, 59),
 ('united kingdom', 44, 93),
 ('united kingdom', 44, 114),
 ('united kingdom', 44, 126),
 ('united kingdom', 44, 147),
 ('united kingdom', 44, 177),
 ('united kingdom', 44, 207),
 ('united kingdom', 44, 295),
 ('united kingdom', 44, 301),
 ('united kingdom', 44, 339),
 ('india', 44, 434),
 ('united kingdom', 44, 493),
 ('united kingdom', 44, 518),
 ('united kingdom', 44, 641),
 ('united kingdom', 44, 648),
 ('united kingdom', 44, 664),
 ('united kingdom', 44, 682),
 ('united kingdom', 44, 699),
 ('united kingdom', 44, 785),
 ('united kingdom', 44, 877),
 ('united kingdom', 44, 953),
 ('unit

In [69]:
#get the first match to prove that the index is the index in the dataframe
df.iloc[ matches[0][2]].Country

'united y'

In [70]:
#they look pretty good, except for the united kingdom, kuwait (kuwait???), india
#lets get all the unique matches in matches
#first all matches
l=[]
for mtch in matches:
    l.append(mtch[0])

# get unique values
l = set(l)
print(l)

{'uniter statez', 'united kingdom', 'kuwait', 'unitef stated', 'united kindom', 'india', 'unite states', 'lithuania', 'united y', 'united  states', 'united sttes'}


In [71]:
#eyeball to get a list of bogus values
dont_sub_these=['lithuania','kuwait','united kingdom', 'india', 'united kindom']

#remove from l
for v in dont_sub_these:
    try:
        l.remove(v)
    except KeyError as e:
        pass
print(l)

#now call the above closure
for v in l:
    fn= fun1(v, 'usa')
    df.Country = df.Country.map(fn)

p_unique()

{'uniter statez', 'unitef stated', 'unite states', 'united y', 'united  states', 'united sttes'}
There are now 193 unique Country entries


## Add a new column to track the changed columns
<mark> HOMEWORK: A better way would be to rename 'Country' to 'Country_original' and add a new column 'Country_computed' so we lose no data. <br>
    For this you need access to 2 columns in the dataframe, so map will not work, you should use apply

In [ ]:
df.head()

In [ ]:
#will receive a row of data and return a single value
def fun2(row):
    #have whole row to operate on
    if row.Country == 'usa':
        return True
    return False


#record whether usa has been found yet or not
df['found_in_db'] = df.apply(fun2, axis=1)

In [ ]:
print((df.found_in_db == True).sum())
print((df.found_in_db == False).sum())

# Lets see if we can do this for other countries.  It will be easier if we have a package that has a list of countries.

<a href="https://pypi.org/project/pycountry/">pycountry</a> is perfect.  It has a list of countries and a fuzzy lookup API. 


In [ ]:
# !conda install -c conda-forge pycountry -y

In [72]:
import pycountry

### Demonstrate Pycountry: The following pycountry call will return a tupple if the country found, otherwise it will throw a LookupError

In [73]:
pycountry.countries.lookup('de')
pycountry.countries.lookup('germany')
pycountry.countries.lookup('usa').alpha_3
pycountry.countries.lookup('United States of America')
# pycountry.countries.lookup('united sates') #fails

Country(alpha_2='DE', alpha_3='DEU', name='Germany', numeric='276', official_name='Federal Republic of Germany')

Country(alpha_2='DE', alpha_3='DEU', name='Germany', numeric='276', official_name='Federal Republic of Germany')

'USA'

Country(alpha_2='US', alpha_3='USA', name='United States', numeric='840', official_name='United States of America')

In [74]:
#lets try a fuzzy search to seee if we get a few more
pycountry.countries.search_fuzzy('wales')

[Country(alpha_2='GB', alpha_3='GBR', name='United Kingdom', numeric='826', official_name='United Kingdom of Great Britain and Northern Ireland'),
 Country(alpha_2='AU', alpha_3='AUS', name='Australia', numeric='036')]

## Lets apply this to the entire DataFrame

In [ ]:
from tqdm.auto import tqdm  #tqdm gives you the neat little status bar, useful for very long running operations
tqdm.pandas()

In [ ]:
def fix(df, i, cont, search_type):
    '''
    df: dataframe
    i: row
    cont: tuple from pycountry lookup hit 
    search_type: unused-what type of pycountry search was used ()
    '''
    # print(f' {search_type} {cont.alpha_3} for {df.loc[i, "Country"]}')
    df.loc[i, 'found_in_db'] = True  #found column
    df.loc[i, 'Country']=cont.alpha_3  #save the 3 letter alpha character for the country
def find_countries(df):
    #record whether pycountry finds result or not
    for i in tqdm(range(len(df))):  # for every row
        try:
            #try the most accurate lookup first
            res = pycountry.countries.lookup(df.loc[i, 'Country']) 
            #if you get here lookup returned result, use it
            fix(df, i, res, "Accurate swap")
        except LookupError as le:
            #lookup above failed, try fuzzy, you may get many results
            try:
                res = pycountry.countries.search_fuzzy(df.loc[i, 'Country'])
                #if you get here search_fuzzy returned result, should you use it?
                #SHOULD CHECK CONFIDENCE, THIS CODE DOES NOT
                fix(df, i, res[0], "Fuzzy swap")
            except LookupError as le:
                pass
    return df


df = find_countries(df)

In [ ]:
df.Country.value_counts()
p_unique()

## Final Tally

In [ ]:
numb_found = df.found_in_db.sum()
print(f' Number countries identified {numb_found}')
print(f' Number countries unidentified {len(df)-numb_found}')
print(f' Number unique countries {df.Country.nunique()}')

In [ ]:
df[df['found_in_db']==False].Country.value_counts()

## Lets see what the top 10 countries are

In [ ]:
df_tt=df.groupby('Country').count().sort_values(by='found_in_db',ascending=False)
df_tt

In [ ]:
# sns.histplot(data=df_tt.iloc[:3,:], x='found_in_db',  hue=df_tt.index[:3]);
# sns.displot(data=df_tt.iloc[:3,:], x='found_in_db',  hue=df_tt.index[:3], kind='hist', fill=True);
df_tmp=df_tt.iloc[:10,:]
sns.barplot(data=df_tmp,x=df_tmp.index, y='found_in_db');

## When finished saved processed data for further evaluation

In [ ]:
# save to feather format
df.to_feather(FILE_FEATHER)